In [1]:
# imports
import time
import sklearn
import pandas as pd
import numpy
import multiprocess as mp

from PIL import Image, ImageOps, ImageEnhance
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
from sklearn import svm
from matplotlib import cm


# directorios
train_dir = '../data/processed/train'
test_dir = '../data/processed/test'

train_csv_path = '../data/processed/train_split.txt'
test_csv_path = '../data/processed/test_split.txt'

train_fd_csv_path = '../data/processed/train_fd.txt'
test_fd_csv_path = '../data/processed/test_fd.txt'

train_hoged_imgs_csv_path = '../data/processed/train_hogged.txt'
test_hoged_imgs_csv_path = '../data/processed/test_hogged.txt'


In [2]:
train_df = pd.read_csv(train_csv_path, delimiter = ',', header='infer')
print('train_df listo')
test_df = pd.read_csv(test_csv_path, delimiter = ',', header='infer')
print('test_df listo')

train_feats_df = pd.read_csv(train_hoged_imgs_csv_path, delimiter = ',', header='infer')
print('train_feats_df listo')
test_feats_df = pd.read_csv(test_hoged_imgs_csv_path, delimiter = ',', header='infer')
print('test_feats_df listo')

train_df listo
test_df listo
train_feats_df listo
test_feats_df listo


In [3]:
train_df['features'] = train_feats_df['features']
test_df['features'] = test_feats_df['features']

In [4]:
train_df['features'][89]

'[0.24339069745014616, 0.04811840899499365, 0.24091163634376572, 0.24339069745014616, 0.24339069745014616, 0.046194626264601926, 0.050585192278755874, 0.04092321573514183, 0.024922200404380136, 0.24339069745014616, 0.15356978012187533, 0.07027520437203767, 0.24339069745014616, 0.14991948697408244, 0.24339069745014616, 0.24339069745014616, 0.0, 0.0, 0.13910294842003937, 0.07483328687614985, 0.23728056911042517, 0.07641480248172272, 0.0788109166252825, 0.18689541503071083, 0.21621113147716614, 0.09031556704244226, 0.11026228793483223, 0.15937241364100507, 0.22218676816238786, 0.17724901162812307, 0.1743089198518813, 0.24339069745014616, 0.24339069745014616, 0.04458218609882026, 0.0, 0.1210756697710997, 0.22685270708804203, 0.13163592961365964, 0.06023803543224172, 0.24856025722081035, 0.12850699544776611, 0.309182460291194, 0.24279977648418805, 0.0, 0.0, 0.251405888421705, 0.08367731311044149, 0.10603632348148065, 0.11294447951459327, 0.08810055411608664, 0.12227613412078361, 0.067463717

In [5]:
numpy.array(train_df['features'][89].replace('[', '').replace(']', '').split(', ')).astype(float)

array([0.2433907 , 0.04811841, 0.24091164, ..., 0.00763156, 0.00863834,
       0.29128588])

In [6]:
train_df['features'] = train_df['features'].apply(lambda x: numpy.array(x.replace('[', '').replace(']', '').split(', ')).astype(float))
train_df.info(verbose=True)
train_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13889 entries, 0 to 13888
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  13889 non-null  int64 
 1   patient_id  13889 non-null  object
 2   image_id    13889 non-null  object
 3   disease     13889 non-null  object
 4   method      13889 non-null  object
 5   ancho       13889 non-null  int64 
 6   alto        13889 non-null  int64 
 7   features    13889 non-null  object
dtypes: int64(3), object(5)
memory usage: 868.2+ KB


,Unnamed: 0,patient_id,image_id,disease,method,ancho,alto,features
0,0,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0....,pneumonia,cohen,1692,1474,"[0.05304781434616521, 0.20525947330046926, 0.2..."
1,1,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5....,pneumonia,cohen,1800,1546,"[0.255982974731789, 0.11452193340504206, 0.065..."
2,2,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10...,pneumonia,cohen,1800,1546,"[0.11804449346683496, 0.0, 0.07003838279215972..."
3,3,7,SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day...,pneumonia,cohen,1772,1800,"[0.16059000086128708, 0.0, 0.04542171066254836..."
4,4,7,SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day...,pneumonia,cohen,1708,1710,"[0.008929087450667601, 0.0, 0.0, 0.02599220965..."
5,5,9,SARS-10.1148rg.242035193-g04mr34g07a-Fig7a-day...,pneumonia,cohen,1800,1508,"[0.06668645292154009, 0.0, 0.32554100540237707..."
6,6,9,SARS-10.1148rg.242035193-g04mr34g07b-Fig7b-day...,pneumonia,cohen,1800,1522,"[0.026395227192158723, 0.0, 0.0345258006625881..."
7,7,10,SARS-10.1148rg.242035193-g04mr34g09a-Fig9a-day...,pneumonia,cohen,1750,1800,"[0.012227901267451175, 0.0, 0.0216160797592121..."
8,8,10,SARS-10.1148rg.242035193-g04mr34g09b-Fig9b-day...,pneumonia,cohen,1800,1779,"[0.10135550723514015, 0.0, 0.11798374449335554..."
9,9,10,SARS-10.1148rg.242035193-g04mr34g09c-Fig9c-day...,pneumonia,cohen,1787,1800,"[0.04496166907898333, 0.01621116018273662, 0.2..."


In [7]:
test_df['features'] = test_df['features'].apply(lambda x: numpy.array(x.replace('[', '').replace(']', '').split(', ')).astype(float))
test_df.info(verbose=True)
test_df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1559 entries, 0 to 1558
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1559 non-null   int64 
 1   patient_id  1559 non-null   object
 2   image_id    1559 non-null   object
 3   disease     1559 non-null   object
 4   method      1559 non-null   object
 5   ancho       1559 non-null   int64 
 6   alto        1559 non-null   int64 
 7   features    1559 non-null   object
dtypes: int64(3), object(5)
memory usage: 97.6+ KB


,Unnamed: 0,patient_id,image_id,disease,method,ancho,alto,features
0,0,8,SARS-10.1148rg.242035193-g04mr34g05x-Fig5-day9...,pneumonia,cohen,1800,1593,"[0.04083452526266057, 0.028069886340383106, 0...."
1,1,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.20284618855773437, 0.03022314367003408, 0.3..."
2,2,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.13175424930510085, 0.03565531022005394, 0.2..."
3,3,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.3264431238262212, 0.08138578706490007, 0.27..."
4,4,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.13175909260528512, 0.035656620913040844, 0...."
5,5,19,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,COVID-19,cohen,1165,1163,"[0.06989347271487874, 0.18086683127545855, 0.2..."
6,6,19,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,COVID-19,cohen,1170,1161,"[0.3004648552842258, 0.06416629797424846, 0.08..."
7,7,19,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,COVID-19,cohen,1165,1165,"[0.09120329615659471, 0.2643528527544757, 0.29..."
8,8,19,1-s2.0-S0929664620300449-gr2_lrg-d.jpg,COVID-19,cohen,1166,1161,"[0.10448392021170066, 0.0, 0.0, 0.002707201281..."
9,9,20,covid-19-pneumonia-15-PA.jpg,COVID-19,cohen,2800,3368,"[0.13259519630601682, 0.011220288227791319, 0...."


In [8]:
print(len(train_df['features'][22]))
print(len(train_df['features'][55]))
print(len(train_df['features'][677]))
print(len(train_df['features'][9000]))
print(len(train_df['features'][10002]))


print(len(test_df['features'][22]))
print(len(test_df['features'][55]))
print(len(test_df['features'][100]))


print(len(train_df['disease']))
print(len(train_df['features']))

86436
86436
86436
86436
86436
86436
86436
86436
13889
13889


In [9]:
print(test_df['disease'].value_counts())

normal       885
pneumonia    594
COVID-19      80
Name: disease, dtype: int64


In [10]:
normal = 'normal'
pneumonia = 'pneumonia'
covid = 'COVID-19'

def disease_to_num(x):
    if x == normal:
        return 0
    elif x == pneumonia:
        return 1
    else:
        return 2

disease_to_num(train_df['disease'][0])

1

In [11]:
train_df['disease'] = train_df['disease'].apply(lambda x: disease_to_num(x))

In [12]:
test_df['disease'] = test_df['disease'].apply(lambda x: disease_to_num(x))

In [13]:
print(train_df['disease'][9])
print(test_df['disease'][92])

1
0


In [14]:
train_set = train_df['features'].to_numpy()
train_set[0]


array([0.05304781, 0.20525947, 0.28578117, ..., 0.06475355, 0.06143061,
       0.09567481])

In [15]:
print(len(train_set))
len(train_set[0])

13889


86436

In [16]:
test_set = test_df['features'].to_numpy()
test_set[0]

array([0.04083453, 0.02806989, 0.05593293, ..., 0.        , 0.        ,
       0.        ])

In [21]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

start = time.time()
clf = RandomForestClassifier(min_samples_leaf=20)
clf.fit(list(train_set), train_df['disease'])
end = time.time()

print("Random Forest", end - start, clf.score(list(train_set), train_df['disease']))
proba = clf.predict_proba(list(train_set))
print("Proba")
print(proba)

y_pred = clf.predict(list(test_set))

print("Accuracy: "+ str(accuracy_score(test_df['disease'], y_pred)))
print('\n')
print(classification_report(test_df['disease'], y_pred))

Random Forest 249.1618320941925 0.8796889624883001
Proba
[[0.26275583 0.48228415 0.25496002]
 [0.30217556 0.42313665 0.27468779]
 [0.30028552 0.47529264 0.22442184]
 ...
 [0.19230491 0.79832631 0.00936878]
 [0.4437528  0.47497303 0.08127417]
 [0.21360022 0.75840097 0.02799881]]
Accuracy: 0.7742142398973701


              precision    recall  f1-score   support

           0       0.78      0.86      0.82       885
           1       0.76      0.74      0.75       594
           2       1.00      0.03      0.05        80

    accuracy                           0.77      1559
   macro avg       0.85      0.54      0.54      1559
weighted avg       0.79      0.77      0.75      1559

